In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from tqdm import tqdm

# Load data
ordini_per_cliente = json.load(open('data/utils/ordini_per_client.json'))       
tot_per_cliente = json.load(open('data/utils/tot_per_client.json')) 
quantity = pd.read_csv('data/utils/quantity_per_client.csv')

# Utils
clienti = list(ordini_per_cliente.keys())               # Lista di clienti
tot = sum(tot_per_cliente.values())                     # Totale ordini per cliente nello storico
norm = {k: v/tot for k,v in tot_per_cliente.items()}    # Probabilità per cliente di comparire nello storico

In [3]:
### Generate synthetic data

## params
num_samples = 10        # Number of samples to generate

# Set to True to generate data starting from today (orders in entrata oggi)
today = True        

# Altrimenti imposta un range
start = '2024-10-01'
end = '2024-12-31'

new_data = []

# Loop to generate synthetic data with progress bar
for _ in tqdm(range(num_samples), desc="Generating data"):

    # Scegli cliente in modo uniforme
    i = np.round(np.random.uniform(0, len(clienti) - 1))
    cliente = clienti[int(i)]
    prob = norm[cliente]
    num = int(np.round(prob * num_samples))                     # Numero di ordini da generare per il cliente
    rows = quantity[quantity['Cliente'] == cliente].shape[0]
    if rows > 0:                                                # Se il cliente ha articoli nel dataset
        articoli = quantity[quantity['Cliente'] == cliente]     # Filtra articoli per cliente
        
        for _ in range(num + 1):
            art_index = int(np.round(np.random.uniform(0, rows - 1)))   # Scegli un articolo in modo uniforme
            art = articoli.iloc[art_index]

            articolo = art['cod articolo']              # Nome dell'articolo
            quantity_mean = art['mean']                 # Media della quantità ordinata                
            quantity_std = art['std']                   # Deviazione standard della quantità ordinata
            giorni = art['mean_differenza_giorni']      # media(consegna - ordine) | articolo, cliente

            # Ensure kg is always positive
            kg = np.round(np.random.normal(quantity_mean, quantity_std))
            if kg <= 0:                                     # Se la quantità è negativa, 
                kg = quantity_mean                              # prendi la media
            
            if today:
                data_inizio = pd.Timestamp.today().date()
            else:
                date_range = pd.date_range(start=start, end=end)
                data_inizio = np.random.choice(date_range)
                data_inizio = pd.Timestamp(data_inizio).date()

            # Calculate the end date
            data_fine = data_inizio + pd.Timedelta(days=giorni)
            data_fine = pd.Timestamp(data_fine).date()

            # Append the generated data to new_data list
            new_data.append({                               # Inserisci come dizionario in una lista di ordini
                'cliente': cliente,
                'cod_articolo': articolo,
                'quantity': kg,
                'data inserimento': data_inizio,
                'data consegna': data_fine
            })

columns = ['cliente', 'cod_articolo', 'quantity', 'data inserimento', 'data consegna']
res = pd.DataFrame(data=new_data, columns=columns)

res.to_csv('data/new_orders.csv', index=False)
res

Generating data: 100%|██████████| 10/10 [00:00<00:00, 1021.78it/s]


,cliente,cod_articolo,quantity,data inserimento,data consegna
0,CALZIFICIO BRAM,X24-2001S+ZN,145.0,2024-10-24,2024-11-04
1,RADICI,2275S+ZN-N00,191.0,2024-10-24,2025-01-09
2,CALZIFICIO NEGRISOLI,1724S+ZN,18.0,2024-10-24,2024-11-19
3,CALZIFICIO SANBA,4424ZF,80.0,2024-10-24,2024-11-19
4,CALZIFICIO LUISA,2246ZN 13500,1200.0,2024-10-24,2024-11-19
5,GIZETA,X17-7895S+ZN,363.0,2024-10-24,2024-11-07
6,GIZETA,4407DN,139.0,2024-10-24,2024-10-29
7,FF,4461SF,205.0,2024-10-24,2024-11-05
8,CALZIFICIO NEGRISOLI,2204ZF1,18.0,2024-10-24,2024-10-25
9,GUSTAV GERSTER,S22-120027V2DN2,5.0,2024-10-24,2024-11-19
